# Stratified Random Sampling

In [8]:
import os
import shutil
import random

# Root dataset directory
root_dir = 'Z:/ExDark'  # Root folder of the dataset
output_dir = 'Z:/ExDark/data'  # Root folder for split datasets

train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

# Supported image formats
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']  # Add more formats if needed

# Make directories for train and val
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Train-val split ratio
train_ratio = 0.8
val_ratio = 0.2

# Get all class folders excluding 'data' folder
class_folders = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d)) and d != 'data']

# Function to check if a file is an image based on the extension
def is_image_file(filename):
    return any(filename.lower().endswith(ext) for ext in image_extensions)

# Iterate over all detected class folders
for class_name in class_folders:
    class_image_dir = os.path.join(root_dir, class_name)

    # Get all files in the folder and filter only image files
    class_files = [f for f in os.listdir(class_image_dir) if is_image_file(f)]
    
    # Create class subdirectories for train and val
    class_train_dir = os.path.join(train_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_val_dir, exist_ok=True)

    # Shuffle files for random split
    random.shuffle(class_files)
    
    # Calculate train/val split
    train_count = int(len(class_files) * train_ratio)
    
    # Split into train and val sets
    train_files = class_files[:train_count]
    val_files = class_files[train_count:]

    # Copy files to train folder
    for file_name in train_files:
        base_name, ext = os.path.splitext(file_name)  # Split base name and extension
        img_src_path = os.path.join(class_image_dir, file_name)
        
        # Copy image file
        img_dst_path = os.path.join(class_train_dir, file_name)
        shutil.copy(img_src_path, img_dst_path)
        
        # Copy corresponding annotation files for each image extension
        for img_ext in image_extensions:
            if ext.lower() == img_ext:
                txt_src_path = os.path.join(class_image_dir, f"{base_name}{img_ext}.txt")
                txt_dst_path = os.path.join(class_train_dir, f"{base_name}{img_ext}.txt")
                if os.path.exists(txt_src_path):
                    shutil.copy(txt_src_path, txt_dst_path)
                else:
                    print(f"Annotation file not found: {txt_src_path}")

    # Copy files to val folder
    for file_name in val_files:
        base_name, ext = os.path.splitext(file_name)
        img_src_path = os.path.join(class_image_dir, file_name)
        
        # Copy image file
        img_dst_path = os.path.join(class_val_dir, file_name)
        shutil.copy(img_src_path, img_dst_path)
        
        # Copy corresponding annotation files for each image extension
        for img_ext in image_extensions:
            if ext.lower() == img_ext:
                txt_src_path = os.path.join(class_image_dir, f"{base_name}{img_ext}.txt")
                txt_dst_path = os.path.join(class_val_dir, f"{base_name}{img_ext}.txt")
                if os.path.exists(txt_src_path):
                    shutil.copy(txt_src_path, txt_dst_path)
                else:
                    print(f"Annotation file not found: {txt_src_path}")

    print(f"Class '{class_name}' split: {train_count} images in train, {len(val_files)} in val.")


Class 'Bicycle' split: 521 images in train, 131 in val.
Class 'Boat' split: 543 images in train, 136 in val.
Class 'Bottle' split: 437 images in train, 110 in val.
Class 'Bus' split: 421 images in train, 106 in val.
Class 'Car' split: 510 images in train, 128 in val.
Class 'Cat' split: 588 images in train, 147 in val.
Class 'Chair' split: 518 images in train, 130 in val.
Class 'Cup' split: 415 images in train, 104 in val.
Class 'Dog' split: 640 images in train, 161 in val.
Class 'Motorbike' split: 402 images in train, 101 in val.
Class 'People' split: 487 images in train, 122 in val.
Class 'Table' split: 404 images in train, 101 in val.


# Conversion

In [18]:
import os
from PIL import Image  # Library to get image size

# Class name to ID mapping (example, modify as needed for ExDark)
class_name_to_id = {
    "Bicycle": 0,
    "Boat": 1,
    "Bottle": 2,
    "Bus": 3,
    "Car": 4,
    "Cat": 5,
    "Chair": 6,
    "Cup": 7,
    "Dog": 8,
    "Motorbike": 9,
    "People": 10,
    "Table": 11
}

# Function to convert the annotations
def convert_annotation(input_path, output_path, image_width, image_height):
    with open(input_path, 'r') as file:
        lines = file.readlines()
    
    with open(output_path, 'w') as output_file:
        for line in lines:
            if line.startswith('%') or line.strip() == "":
                continue  # Skip header or empty lines

            parts = line.split()
            class_name = parts[0]  # Object class name
            l = float(parts[1])  # Left (x)
            t = float(parts[2])  # Top (y)
            w = float(parts[3])  # Width
            h = float(parts[4])  # Height

            # Convert class name to class ID
            class_id = class_name_to_id.get(class_name, -1)
            if class_id == -1:
                print(f"Warning: Class '{class_name}' not found in mapping.")
                continue

            # YOLO format requires center_x, center_y, width, height normalized
            x_center = (l + w / 2) / image_width
            y_center = (t + h / 2) / image_height
            width = w / image_width
            height = h / image_height

            # Write to the new YOLO format file
            output_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Directory paths
root_dir = 'Z:/ExDark/data'  # Base directory containing 'train' and 'val'
folders_to_process = ['train', 'val']  # Folders to process

# Supported image formats
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']

# Helper function to find the corresponding image file
def find_image_file(base_filename, directory):
    # Ensure base_filename doesn't contain an extension
    base_filename_without_ext = os.path.splitext(base_filename)[0]
    
    for ext in image_extensions:
        image_path = os.path.join(directory, base_filename_without_ext + ext)
        if os.path.isfile(image_path):  # Check if file exists
            print(f"Found image file: {image_path}")  # Debug print
            return image_path
    print(f"Image file not found for base filename '{base_filename_without_ext}'")  # Debug print
    return None

# Loop through 'train' and 'val' directories
for folder in folders_to_process:
    folder_path = os.path.join(root_dir, folder)
    print(f"Processing folder: {folder_path}")  # Debug print

    for class_name in os.listdir(folder_path):
        class_dir = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_dir):
            print(f"Skipping non-directory: {class_dir}")  # Debug print
            continue  # Skip if it's not a directory

        # Process each file in the class folder
        for annotation_file in os.listdir(class_dir):
            if annotation_file.endswith('.txt'):
                annotation_path = os.path.join(class_dir, annotation_file)

                # The base image filename without the '.txt'
                base_filename = annotation_file.replace('.txt', '')
                print(f"Processing annotation file: {annotation_path}")  # Debug print

                # Find the image with any supported format
                image_path = find_image_file(base_filename, class_dir)

                if image_path is None:
                    print(f"Warning: Image file for '{base_filename}' not found.")
                    continue
                
                # Get the resolution (width, height) of the image
                try:
                    with Image.open(image_path) as img:
                        image_width, image_height = img.size
                except Exception as e:
                    print(f"Error opening image file {image_path}: {e}")
                    continue

                # Convert and overwrite the existing annotation file in YOLO format
                convert_annotation(annotation_path, annotation_path, image_width, image_height)
                print(f"Converted: {annotation_path} with resolution {image_width}x{image_height}")

print("Conversion completed!")


Processing folder: Z:/ExDark/data\train
Processing annotation file: Z:/ExDark/data\train\Bicycle\2015_00001.png.txt
Found image file: Z:/ExDark/data\train\Bicycle\2015_00001.png
Converted: Z:/ExDark/data\train\Bicycle\2015_00001.png.txt with resolution 500x375
Processing annotation file: Z:/ExDark/data\train\Bicycle\2015_00002.png.txt
Found image file: Z:/ExDark/data\train\Bicycle\2015_00002.png
Converted: Z:/ExDark/data\train\Bicycle\2015_00002.png.txt with resolution 500x341
Processing annotation file: Z:/ExDark/data\train\Bicycle\2015_00003.png.txt
Found image file: Z:/ExDark/data\train\Bicycle\2015_00003.png
Converted: Z:/ExDark/data\train\Bicycle\2015_00003.png.txt with resolution 500x375
Processing annotation file: Z:/ExDark/data\train\Bicycle\2015_00004.jpg.txt
Found image file: Z:/ExDark/data\train\Bicycle\2015_00004.jpg
Converted: Z:/ExDark/data\train\Bicycle\2015_00004.jpg.txt with resolution 640x360
Processing annotation file: Z:/ExDark/data\train\Bicycle\2015_00005.jpg.txt


# Special format remover

In [3]:
import os

# Define the paths to the data/train and data/val directories
directories = ['data/train', 'data/val']

# Define the image extensions that need to be removed
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']

def rename_txt_files(directory):
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.txt'):
                # Remove image extension from the filename if present before '.txt'
                new_filename = filename
                for ext in image_extensions:
                    if ext in filename:
                        new_filename = filename.replace(ext, '')
                        break  # Exit loop after finding the first matching extension
                
                # Create the full path for the current and new filenames
                old_file = os.path.join(foldername, filename)
                new_file = os.path.join(foldername, new_filename)
                
                # Rename the file
                os.rename(old_file, new_file)

# Run the renaming process for both the data/train and data/val directories
for directory in directories:
    rename_txt_files(directory)

print("Renaming complete!")


Renaming complete!


# Boxer

In [6]:
from PIL import Image, ImageDraw, ImageFont

# Function to parse annotations and draw bounding boxes with labels
def draw_bounding_boxes(image_path, annotations, font_path='arial.ttf', font_size=30):
    # Open an image
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)

    # Load a TrueType font (you can specify a font path if needed)
    try:
        font = ImageFont.truetype(font_path, font_size)  # Increase font size
    except IOError:
        print(f"Font file '{font_path}' not found. Using default font.")
        font = ImageFont.load_default()

    # Loop through each annotation
    for annotation in annotations:
        # Parse the annotation details
        parts = annotation.split()
        obj_class = parts[0]
        l, t, w, h = map(int, parts[1:5])
        
        # Define the bounding box (left, top, right, bottom)
        bbox = (l, t, l + w, t + h)
        
        # Draw the bounding box on the image with a thicker line (width=5)
        draw.rectangle(bbox, outline="red", width=15)
        
        # Get the size of the label
        text_size = draw.textsize(obj_class, font=font)
        
        # Draw label background
        text_background = [(l, t - text_size[1] - 10), (l + text_size[0], t)]
        draw.rectangle(text_background, fill="red")  # Background for the label
        
        # Draw the label above the box
        draw.text((l, t - text_size[1] - 10), obj_class, fill="white", font=font)

    # Show the image with bounding boxes and labels
    img.show()

# Example annotations from the bbGt format
annotations = [
    "People 1578 1061 391 1146 0 0 0 0 0 0 0",
    "Car 1841 1214 446 412 0 0 0 0 0 0 0",
    "Car 2321 1273 276 136 0 0 0 0 0 0 0"
]

# Image path to be annotated
image_path = 'Z:/ExDark/People/2015_06691.JPG'

# Call the function to draw bounding boxes on the image
draw_bounding_boxes(image_path, annotations, font_size=72)


C:\Users\User\AppData\Local\Temp\ipykernel_20488\3157494363.py:30: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(obj_class, font=font)
C:\Users\User\AppData\Local\Temp\ipykernel_20488\3157494363.py:30: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(obj_class, font=font)
C:\Users\User\AppData\Local\Temp\ipykernel_20488\3157494363.py:30: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(obj_class, font=font)
